I read in the dataframe that I previously prepared for linear regression:

In [1]:
import pandas as pd
df = pd.read_pickle('EU_industry_production_dataframe_forregression.pkl')
df.info()
print(df.head())

<class 'pandas.core.frame.DataFrame'>
Float64Index: 212 entries, -10.0013689254 to 7.58110882957
Data columns (total 36 columns):
AT      211 non-null float64
BA      139 non-null float64
BE      211 non-null float64
BG      211 non-null float64
CY      211 non-null float64
CZ      211 non-null float64
DE      211 non-null float64
DK      211 non-null float64
EA19    211 non-null float64
EE      211 non-null float64
EL      211 non-null float64
ES      211 non-null float64
EU28    211 non-null float64
FI      211 non-null float64
FR      211 non-null float64
HR      211 non-null float64
HU      211 non-null float64
IE      211 non-null float64
IT      211 non-null float64
LT      211 non-null float64
LU      211 non-null float64
LV      211 non-null float64
ME      91 non-null float64
MK      211 non-null float64
MT      211 non-null float64
NL      211 non-null float64
NO      211 non-null float64
PL      211 non-null float64
PT      211 non-null float64
RO      211 non-null float64
R

Note that Bosnia and Herzegovina ('BA') and Montenegro ('ME') have many missing values, since their time series do not start in 2000, but later.

The fitting routine for the linear model in scikit-learn does not accept NaN values though, so I need to account for that. Because only two countries are affected (which are not even in the EU), I simply drop them:

In [2]:
df.drop(['BA','ME'], axis=1, inplace=True)

In addition, all the other time series have only 211 non-null entries out of 212 values in total. These single NaN entries are the latest data point for each country. I drop them as well and get nice NaN-free time series:

In [3]:
df.drop(df.index[-1], inplace=True)
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Float64Index: 211 entries, -10.0013689254 to 7.49623545517
Data columns (total 34 columns):
AT      211 non-null float64
BE      211 non-null float64
BG      211 non-null float64
CY      211 non-null float64
CZ      211 non-null float64
DE      211 non-null float64
DK      211 non-null float64
EA19    211 non-null float64
EE      211 non-null float64
EL      211 non-null float64
ES      211 non-null float64
EU28    211 non-null float64
FI      211 non-null float64
FR      211 non-null float64
HR      211 non-null float64
HU      211 non-null float64
IE      211 non-null float64
IT      211 non-null float64
LT      211 non-null float64
LU      211 non-null float64
LV      211 non-null float64
MK      211 non-null float64
MT      211 non-null float64
NL      211 non-null float64
NO      211 non-null float64
PL      211 non-null float64
PT      211 non-null float64
RO      211 non-null float64
RS      211 non-null float64
SE      211 non-null float64


Next, I import the "linear_model" class from scikit-learn:

In [4]:
from sklearn import linear_model

Before actually performing the linear regression, I need to bring the dataset in a form that is recognized by scikit-learn. The fitting method takes two parameters: the input X and the output y.

X can be identified with the dependent variables, which is just one in this case, namely the time.

y is the outcome, that is the normalized industry production index.

The input X has to be a 2D array and is the same for all countries, so I assign it right away:

In [5]:
X = df.index.values                # Get numpy array of index (time stamps in years)
print(X.shape)

X = X.reshape(-1, 1)               # Transform numpy array from 1D to 2D
print(X.shape)

(211,)
(211, 1)


The output y is different for each country, so I will loop over all countries and perform separate linear regressions.

I can get the country codes as a list by extracting all the column names from the dataframe:

In [6]:
country_list = df.columns.values   # Get list of all country codes
print(country_list)

['AT' 'BE' 'BG' 'CY' 'CZ' 'DE' 'DK' 'EA19' 'EE' 'EL' 'ES' 'EU28' 'FI' 'FR'
 'HR' 'HU' 'IE' 'IT' 'LT' 'LU' 'LV' 'MK' 'MT' 'NL' 'NO' 'PL' 'PT' 'RO' 'RS'
 'SE' 'SI' 'SK' 'TR' 'UK']


Now I loop over all countries. In each step, I assign the normalized industry production values to y, perform the linear regression, and store the slope and intercept values in numpy arrays:

In [7]:
import numpy as np

slope = np.empty(len(country_list))         # Create empty array for slopes from linear regression
intercept = np.empty(len(country_list))     # Create empty array for intercept from linear regression

# Loop over all countries:
for i, country in enumerate(country_list):
    y = df[country].values                  # Get industry production index values for selected country
    regr = linear_model.LinearRegression()  # Instantiate linear regression object
    regr.fit(X, y)                          # Perform linear regression (fit straight line)
    slope[i] = regr.coef_[0]                # Store slope in array (first and only coefficient)
    intercept[i] = regr.intercept_          # Store intercept in array

Let's have a look at the parameter values that I obtained from the linear regression:

In [8]:
print(slope)
print(intercept)

[  1.69755062e-02   2.27397515e-02   2.33266960e-02  -2.73036128e-02
   3.01791696e-02   1.10648741e-02  -2.54430214e-03  -2.06856083e-03
   4.36360429e-02  -3.08404991e-02  -2.61402287e-02   3.36143245e-05
  -3.74731167e-03  -1.69121515e-02   8.05852423e-04   2.80566577e-02
   3.92839139e-02  -2.34791885e-02   3.66338076e-02  -6.37034263e-03
   2.77396054e-02   1.52164653e-02  -8.32050455e-03   5.89524088e-03
   6.97164004e-03   4.64402255e-02  -2.25316217e-02   3.60324141e-02
  -6.70048827e-04  -9.21470009e-03   9.93230550e-03   5.91275559e-02
   3.89830067e-02  -7.94040227e-03]
[ 0.97951101  0.93506694  1.00117521  0.90635968  0.98434731  0.98850886
  1.07562782  1.00401122  1.06813739  1.01401165  1.03302358  1.00149184
  0.97096251  1.03319856  0.95866234  0.98838774  1.04417995  1.00206516
  1.02512619  0.99293733  1.06671794  1.07534489  0.9701639   0.96239976
  0.96680512  0.92858528  1.04249824  1.02776252  1.01763429  0.974905
  0.97854261  0.99558271  0.98038008  1.00006333]

As a simple measure to check for the robustness of the slope values, I also calculate the slope using an alternative method, namely the difference between the last and the first production index values of the time series, divided by the length of the time series in years.

To reduce the influence of the short-time fluctuations, I average over the five first and last values instead of just taking one value each:

In [9]:
slope_alt = np.empty(len(country_list))     # Create empty array for slopes from difference end of time series minus beginning

# Loop over all countries:
for i, country in enumerate(country_list):
    y = df[country].values                  # Get industry production index values for selected country
    slope_alt[i] = (y[-5:].mean() - y[:5].mean())/(X[-1,0] - X[0,0])      # Store slope in array

Let me also print the alternative slope values:

In [10]:
print(slope_alt)

[ 0.01540618  0.0203894   0.02858958 -0.01814415  0.03178366  0.00758084
  0.00022828 -0.00215899  0.04310238 -0.02575114 -0.02014504 -0.00021196
 -0.0016687  -0.01616444  0.00645503  0.0299987   0.04527303 -0.02057385
  0.03709763 -0.00015542  0.03415784  0.00874983 -0.0112554   0.0037613
 -0.00294682  0.04093304 -0.01623621  0.03449866  0.00147644 -0.00471612
  0.01575481  0.05198007  0.03306995 -0.01023881]


Let's now create a new dataframe containing the two slope estimates and the intercept from the linear regression:

In [11]:
# Put data in dictionary:
slope_dict = {'country_code':country_list, 'slope':slope, 'intercept':intercept, 'slope_alt':slope_alt}

# Construct dataframe from dictionary:
df_slopes = pd.DataFrame(slope_dict)

# Choose country_code column as index:
df_slopes.set_index('country_code', inplace=True)

print(df_slopes.info())
print(df_slopes.head())

<class 'pandas.core.frame.DataFrame'>
Index: 34 entries, AT to UK
Data columns (total 3 columns):
intercept    34 non-null float64
slope        34 non-null float64
slope_alt    34 non-null float64
dtypes: float64(3)
memory usage: 1.1+ KB
None
              intercept     slope  slope_alt
country_code                                
AT             0.979511  0.016976   0.015406
BE             0.935067  0.022740   0.020389
BG             1.001175  0.023327   0.028590
CY             0.906360 -0.027304  -0.018144
CZ             0.984347  0.030179   0.031784


This dataframe contains the slopes for each country. I will visually explore and analyze them in the next project.

For now, I end by saving the dataframe in a pickled file:

In [12]:
df_slopes.to_pickle('EU_industry_production_slopes.pkl')